# The objective of a data warehouse is to enable analyzing historical data

In [ ]:
! python ../setup.py -db_file tpch.db

In [ ]:
import duckdb
import pandas as pd

%load_ext sql
conn = duckdb.connect("tpch.db")
%sql conn --alias duckdb

In [ ]:
%%sql
show tables;

## Understanding how your business works is critical

- data flow from product/upstream team
- create a conceptual data flow diagram
- Use tpch as example

## Data modeling refers to how your data is stored for historical analytics


The term analytical querying usually refers to aggregating numerical (spend, count, sum, avg) data from the fact table for specific dimension attribute(s) (e.g., name, nation, date, month) from the dimension tables. Some examples of analytical queries are 

1. Who are the top 10 suppliers (by totalprice) in the past year?

2. What are the average sales per nation per year?

3. How do customer market segments perform (sales) month-over-month?

The questions above ask about **historically aggregating data from the fact tables for one or more business entities(dimensions)**. 
Consider the example analytical question below and notice the facts and dimensions.

![Analytical query](./images/im_2.png)
add: images

When we dissect the above analytical query, we see that it involves:

1. Joining the fact data with dimension table(s) to get the dimension attributes such as name, region, & brand. In our example, we join the orders fact table with the customer dimension table.

2. **Modifying granularity** (aka rollup, Group by) of the joined table to the dimension(s) in question. In our example, this refers to `GROUP BY custkey, YEAR(orderdate).`



### Kimball dimensional modeling is by far the most popular among all options

### Real life events are called facts

 **Facts**: Each row in a fact table represents a business process that occurred. E.g., In our data warehouse, each row in the `orders` fact table will represent an individual order, and each row in the `lineitem` fact table will represent an item sold as part of an order. Each fact row will have a unique identifier; in our case, it's `orderkey` for orders and a combination of `orderkey & linenumber` for lineitem.


A fact table's **grain (aka granularity, level)** refers to what a row in a fact table represents. For example, in our checkout process, we can have two fact tables, one for the order and another for the individual items in the order. The items table will have one row per item purchased, whereas the order table will have one row per order made.


In [ ]:
%%sql

-- calculating the totalprice of an order (with orderkey = 1) from it's individual items
SELECT
    l_orderkey,
    round( sum(l_extendedprice * (1 - l_discount) * (1 + l_tax)),
        2
    ) AS totalprice
    -- Formula to calculate price paid after discount & tax
FROM
    lineitem
WHERE
    l_orderkey = 1
GROUP BY
    l_orderkey;

/*
 orderkey | totalprice
----------+------------
        1 |  172799.56
*/


In [ ]:
%%sql
-- The totalprice of an order (with orderkey = 1)
SELECT
    o_orderkey,
    o_totalprice
FROM
    orders
WHERE
    o_orderkey = 1;

### Dimension = Someone/something that interacts with your business

 **Dimension**: Each row in a dimension table represents a business entity that is important to the business. For example, A car parts seller's data warehouse will have a `customer` dimension table, where each row will represent an individual customer. Other examples of dimension tables in a car parts seller's data warehouse would be `supplier` & `part` tables. Techniques such as [SCD2](https://www.startdataengineering.com/post/how-to-join-fact-scd2-tables/#what-is-an-scd2-table-and-why-use-it) are used to store data whose values can change over time (e.g., customers address).


In [ ]:
%%sql
select * from customer limit 5;

In [ ]:
%%sql
select * from supplier limit 2;

## Most tech companies follow the 3-hop architecture

Most data teams have their version of the 3-hop architecture. For example, dbt has its own version (stage, intermediate, mart), and Spark has medallion (bronze, silver, gold) architecture.

You may be wondering why we need this data flow architecture when we have the results easily with a simple query shown here.

While this is a simple example, in most real-world projects you want to have a standard, cleaned and modelled dataset(bronze) that can be use to create specialized dataset for end-users(gold). See below for how our data will flow:
                                                                                                                                                                         add: data 3-hop arch image                              

### Bronze: Extract raw data and confine it to standard names and data types

Since our dataset has data from customer, nation, region, order, and lineitem input datasets, we will bring those data into bronze tables. We will keep their names the same as the input datasets.

Let's explore the input datasets and create our bronze datasets.

In [ ]:
%%sql
DROP SCHEMA IF EXISTS bronze CASCADE;
CREATE SCHEMA IF NOT EXISTS bronze;

In [ ]:
%%sql
-- Remove 'c_' prefix and rename 'custkey' to 'customer_key'
DROP TABLE IF EXISTS bronze.customer;
CREATE TABLE bronze.customer AS 
SELECT 
    c_custkey AS customer_key,
    c_name AS name,
    c_address AS address,
    c_nationkey AS nationkey,
    c_phone AS phone,
    c_acctbal AS acctbal,
    c_mktsegment AS mktsegment,
    c_comment AS comment
FROM customer;


In [ ]:
%%sql

DROP TABLE IF EXISTS bronze.nation;
CREATE TABLE bronze.nation AS 
SELECT 
    n_nationkey AS nationkey,
    n_name AS name,
    n_regionkey AS regionkey,
    n_comment AS comment
FROM nation;

-- Region Table
DROP TABLE IF EXISTS bronze.region;
CREATE TABLE bronze.region AS 
SELECT 
    r_regionkey AS regionkey,
    r_name AS name,
    r_comment AS comment
FROM region;

-- Orders Table
DROP TABLE IF EXISTS bronze.orders;
CREATE TABLE bronze.orders AS 
SELECT 
    o_orderkey AS orderkey,
    o_custkey AS custkey,
    o_orderstatus AS orderstatus,
    o_totalprice AS totalprice,
    o_orderdate AS orderdate,
    o_orderpriority AS orderpriority,
    o_clerk AS clerk,
    o_shippriority AS shippriority,
    o_comment AS comment
FROM orders;

-- Lineitem Table
DROP TABLE IF EXISTS bronze.lineitem;
CREATE TABLE bronze.lineitem AS 
SELECT 
    l_orderkey AS orderkey,
    l_partkey AS partkey,
    l_suppkey AS suppkey,
    l_linenumber AS linenumber,
    l_quantity AS quantity,
    l_extendedprice AS extendedprice,
    l_discount AS discount,
    l_tax AS tax,
    l_returnflag AS returnflag,
    l_linestatus AS linestatus,
    l_shipdate AS shipdate,
    l_commitdate AS commitdate,
    l_receiptdate AS receiptdate,
    l_shipinstruct AS shipinstruct,
    l_shipmode AS shipmode,
    l_comment AS comment
FROM lineitem;

### Silver: Model data for analytics

In the silver layer, the datasets are modeled using one of the popular styles (e.g., Kimball, Data Vault, etc.). We will use Kimball's dimensional model, as it is the most commonly used one and can account for many use cases.

**Data modeling**

We will create the following datasets

1. **dim_customer**: A customer level table with all the necessary attributes of a customer. We will join nation and region data to the cleaned_customer_df to get all the attributes associated with a customer.
2. **fct_orders**: An order level fact(an event that happened) table. This will be the same as cleaned_orders_df since the orders table has all the necessary details about the order and how it associates with dimension tables like customer_key.
3. **fct_lineitem**: A lineitem (items that are part of an order) fact table. This table will be the same as cleaned_lineitem_df since the lineitem table has all the lineitem level details and keys to associate with dimension tables like partkey and suppkey.


In [ ]:
%%sql
-- Create Customer Dimension Table by Left-Joining Nation and Region Data
DROP TABLE IF EXISTS dim_customer;
CREATE TABLE dim_customer AS
SELECT 
    c.customer_key,
    c.name AS customer_name,
    c.address,
    c.phone,
    c.acctbal,
    c.mktsegment,
    n.name AS nation_name,
    n.comment AS nation_comment,
    r.name AS region_name,
    r.comment AS region_comment
FROM bronze.customer AS c
LEFT JOIN bronze.nation AS n ON c.nationkey = n.nationkey
LEFT JOIN bronze.region AS r ON n.regionkey = r.regionkey;


In [ ]:
%%sql
-- Fact Orders Table (Direct Data from the App)
DROP TABLE IF EXISTS fct_orders;
CREATE TABLE fct_orders AS
SELECT 
    o.orderkey,
    o.custkey,
    o.orderstatus,
    o.totalprice,
    o.orderdate,
    o.orderpriority,
    o.clerk,
    o.shippriority,
    o.comment
FROM bronze.orders AS o;

-- Fact LineItem Table (Direct Data from the App)
DROP TABLE IF EXISTS fct_lineitem;
CREATE TABLE fct_lineitem AS
SELECT 
    l.orderkey,
    l.partkey,
    l.suppkey,
    l.linenumber,
    l.quantity,
    l.extendedprice,
    l.discount,
    l.tax,
    l.returnflag,
    l.linestatus,
    l.shipdate,
    l.commitdate,
    l.receiptdate,
    l.shipinstruct,
    l.shipmode,
    l.comment
FROM bronze.lineitem AS l;

### Gold: Create tables for end-users

The gold layer contains datasets required for the end user. The user-required datasets are fact tables joined with dimension tables aggregated to the necessary grain. In real-world projects, multiple teams/users ask for datasets with differing grains from the same underlying fact and dimension tables. While you can join the necessary tables and aggregate them individually for each ask, it leads to repeated code and joins.

To avoid this issue, companies typically do the following:

1. **OBT**: This is a fact table with multiple dimension tables left joined with it.
2. **pre-aggregated table**: The OBT table rolled up to the end user/team requested grain. The pre-aggregated dataset will be the dataset that the end user accesses. By providing the end user with the exact columns they need, we can ensure that all the metrics are in one place and issues due to incorrect metric calculations by end users are significantly reduced. These tables act as our end-users SOT (source of truth).

#### OBT: Join the fact table with all its dimensions

In our example, we have two fact tables, fct_orders and fct_lineitem. Since we only have one dimension, dim_customer, we can join fct_orders and dim_customer to create wide_orders. For our use case, we can keep fct_lineitem as wide_lineitem.

That said, we can easily see a case where we might need to join parts and supplier data with fct_lineitem to get wide_lineitem. But since our use case doesn't require this, we can skip it!

Let's create our OBT tables


In [ ]:
%%sql
-- order level OBT wide_orders
DROP TABLE IF EXISTS wide_orders;
CREATE TABLE wide_orders AS
SELECT o.*,
    c.*
FROM fct_orders o
LEFT JOIN dim_customer c 
ON o.custkey = c.customer_key;

-- lineitem level OBT wide_lineitem
-- For our use case, we don't need more information at a lineitem level
DROP TABLE IF EXISTS wide_lineitem;
CREATE TABLE wide_lineitem AS
SELECT * FROM fct_lineitem;

#### Pre-aggregated tables: Aggregate OBTs to stakeholder-specific grain

According to our data requirements, we need data from customer, orders, and lineitem. Since we already have customer and order data in wide_orders, we can join that with wide_lineitem to get the necessary data.

We can call the final dataset customer_outreach_metrics (read this article that discusses the importance of naming).

Let's create our final dataset in Python


In [ ]:
%%sql
DROP TABLE IF EXISTS order_lineitem_metrics;
CREATE TABLE order_lineitem_metrics AS
-- Create Order Lineitem Metrics by Counting Lineitems per Order
SELECT 
    orderkey AS order_key,
    COUNT(linenumber) AS num_lineitems
FROM wide_lineitem
GROUP BY orderkey;

In [ ]:
%%sql
DROP TABLE IF EXISTS customer_outreach_metrics;
CREATE TABLE customer_outreach_metrics AS
-- Create Wide Orders with Order Lineitem Metrics, Aggregated by Customer
SELECT 
    o.customer_key,
    o.customer_name,
    MIN(o.totalprice) AS min_order_value,
    MAX(o.totalprice) AS max_order_value,
    AVG(o.totalprice) AS avg_order_value,
    AVG(m.num_lineitems) AS avg_num_items_per_order
FROM wide_orders AS o
LEFT JOIN (
    SELECT 
        orderkey AS order_key,
        COUNT(linenumber) AS num_lineitems
    FROM wide_lineitem
    GROUP BY orderkey
) AS m ON o.orderkey = m.order_key
GROUP BY o.customer_key, o.customer_name;

In [ ]:
%%sql
select * from customer_outreach_metrics limit 4;